In [ ]:
pth = 'mystical-accord-330011-2b79748d3113.json'

from google.oauth2 import service_account
import pandas_gbq
credentials = service_account.Credentials.from_service_account_file(pth)

id = 'mystical-accord-330011'
sql = 'select * from london_house_prices.london_house_prices'

df = pandas_gbq.read_gbq(sql, project_id= id, credentials=credentials)

In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [ ]:
# pth = 'dataset.csv'
# df = pd.read_csv(pth)

In [ ]:
def clean(x):
  return x[:10]

df['date'] = df['date'].apply(clean)

In [ ]:
df.drop_duplicates(inplace=True)
df.shape

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.head()

,address,type,bedrooms,latitude,longitude,area,price,tenure,is_newbuild,date
0,"Flat 29, Mulberry Court, 1, School Mews, Londo...",Flat,0,51.51061,-0.05992,E1,500000,Leasehold,1,2011-03-02
1,"Flat 29, Mulberry Court, 1, School Mews, Londo...",Flat,0,51.51061,-0.05992,E1,683200,Leasehold,0,2016-01-05
2,"Flat 29, Mulberry Court, 1, School Mews, Londo...",Flat,0,51.51061,-0.05992,E1,640000,Leasehold,0,2020-07-15
3,"Flat 205, Brody House, Strype Street, London, ...",Flat,0,51.51736,-0.07621,E1,136000,Leasehold,1,1999-09-30
4,"Flat 205, Brody House, Strype Street, London, ...",Flat,0,51.51736,-0.07621,E1,199950,Leasehold,0,2004-05-04


In [ ]:
cols = ['address',
        'type',	
        'bedrooms',	'latitude', 'longitude', 'area', 'tenure'
        ]

df_new = df.groupby(cols, as_index = False).agg({'price':['mean'],
                                               'is_newbuild':['max'],
                                               'date':['max']
                                               })

cols_add =  ['price', 'is_newbuild', 'date']
df_new.columns = cols + cols_add
df_new.shape

(113018, 10)

In [ ]:
df_new.head()

,type,bedrooms,latitude,longitude,area,tenure,price,is_newbuild,year_sold,total_days,address1,address2,address3,address4,address5,address6,address7
0,Semi-Detached,1,51.54744,-0.14633,NW5,Leasehold,1850000.0,1,2017,1735,NaN,007,Brinsmead Apartments,25a,Ryland Road,London,Greater London NW5 3EH
1,Flat,4,51.46522,-0.01289,SE13,Leasehold,402500.0,1,2019,797,NaN,NaN,NaN,032 River Mill One,Station Road,London,Greater London SE13 5FL
2,Flat,1,51.46503,-0.01218,SE13,Leasehold,437500.0,1,2018,1079,NaN,NaN,NaN,062 Brick Kiln One,Station Road,London,Greater London SE13 5FP
3,Flat,1,51.46504,-0.01276,SE13,Leasehold,328000.0,0,2019,903,NaN,NaN,NaN,076 River Mill One,Station Road,London,Greater London SE13 5FL
4,Semi-Detached,9,51.54500,-0.15681,NW3,Freehold,3350000.0,0,2019,1036,NaN,NaN,NaN,1 - 2,Eton Villas,London,Greater London NW3 4SX


In [ ]:
df_new['date'] = pd.to_datetime(df_new['date'])
df_new['year_sold'] = df_new.date.dt.year
df_new['total_days']=(pd.to_datetime('today')-df_new['date']).dt.days
df_new.drop(["date"],axis=1,inplace=True)

In [ ]:
def new_address(x):
  x = x.split(', ')
  len_x = len(x)

  if len_x < 7:
    x = ['']*(8 - len_x) + x

  return x[-7:]


addr_cols = ['address1', 'address2', 'address3', 'address4', 'address5', 
             'address6', 'address7']
             
df_new[addr_cols] = ''

for i in df_new.index:
  address = new_address(df_new.loc[i, 'address'])
  df_new.loc[i, addr_cols] = address

In [ ]:
df_new.drop(["address"],axis=1,inplace=True)

In [ ]:
cat_cols = ['type', 'bedrooms', 'area', 'tenure', 'is_newbuild'] + addr_cols

In [ ]:
lbl_encoders={}
pth = 'encode/'
for feature in cat_cols:
    lbl_encoders[feature]=LabelEncoder()
    df_new[feature]=lbl_encoders[feature].fit_transform(df_new[feature].astype(str))
    with open(f'{pth}{feature}', 'wb') as f0:
      pickle.dump(lbl_encoders[feature], f0)

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler

In [ ]:
class MakeDataset(torch.utils.data.Dataset):

  def __init__(self, X, y, scale_data=True):
    if not torch.is_tensor(X) and not torch.is_tensor(y):
      X = StandardScaler().fit_transform(X)
      self.X = torch.from_numpy(X)
      self.y = torch.from_numpy(y)

  def __len__(self):
      return len(self.X)

  def __getitem__(self, i):
      return self.X[i], self.y[i]

In [ ]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(16, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 1)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

In [ ]:
torch.manual_seed(42)
y = df_new.pop('price')
X = df_new.to_numpy()
y = y.to_numpy()

In [ ]:
dataset = MakeDataset(X, y)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=1)

In [ ]:
mlp = MLP()
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

In [ ]:

for epoch in range(0, 5):
  print(f'Starting epoch {epoch+1}')
  
  current_loss = 0.0
  
  for i, data in enumerate(trainloader, 0):
    
    inputs, targets = data
    inputs, targets = inputs.float(), targets.float()
    targets = targets.reshape((targets.shape[0], 1))
    
    optimizer.zero_grad()
    
    outputs = mlp(inputs)
    
    loss = loss_function(outputs, targets)
    
    loss.backward()
    
    optimizer.step()
    
    current_loss += loss.item()
    if i % 10 == 0:
        print('Loss after mini-batch %5d: %.3f' %
              (i + 1, current_loss / 500))
        current_loss = 0.0


Выходные данные были обрезаны до нескольких последних строк (5000).
Loss after mini-batch  6601: 11922.403
Loss after mini-batch  6611: 16330.532
Loss after mini-batch  6621: 15425.421
Loss after mini-batch  6631: 15560.170
Loss after mini-batch  6641: 12142.346
Loss after mini-batch  6651: 10928.876
Loss after mini-batch  6661: 10264.877
Loss after mini-batch  6671: 11173.908
Loss after mini-batch  6681: 12745.382
Loss after mini-batch  6691: 10640.818
Loss after mini-batch  6701: 12444.499
Loss after mini-batch  6711: 14375.240
Loss after mini-batch  6721: 11567.954
Loss after mini-batch  6731: 10972.651
Loss after mini-batch  6741: 12359.129
Loss after mini-batch  6751: 12000.713
Loss after mini-batch  6761: 12265.405
Loss after mini-batch  6771: 11271.544
Loss after mini-batch  6781: 14260.093
Loss after mini-batch  6791: 12893.131
Loss after mini-batch  6801: 12783.636
Loss after mini-batch  6811: 12163.982
Loss after mini-batch  6821: 11954.290
Loss after mini-batch  6831: 11883.

In [ ]:
PATH = 'torch_model.bin'
torch.save(mlp, PATH)